In [0]:
%sql
-- Magic Commands

-- %python
-- %sql
-- %md
-- %md-sandbox
-- %run ./Includes/Setup
-- %fs ls 'databricks-datasets' #
-- files = dbutils.fs.ls('/databricks-datasets')
-- print(files)
-- display(files)
-- dbutils.help() 
-- dbutils.fs.help()

In [0]:
%sql
USE CATALOG hive_metastore;

-- DESCRIBE DETAIL table_name;
-- DESCRIBE HISTORY table_name;
-- DESCRIBE DATABASE EXTENDED new_default;
-- RESTORE TABLE employees TO VERSION AS OF 5;
-- OPTIMIZE employees ZORDER BY id;
-- VACUUM employees [RETAIN 0 HOURS];

-- %fs ls 'dbfs:/user/hive/warehouse/employees/_delta_log'
-- %fs head 'dbfs:/user/hive/warehouse/employees/_delta_log/00000000000000000005.json'
-- SET spark.databricks.delta.retentionDurationCheck.enabled = false;

In [0]:
%sql
CREATE TABLE employees
  (id INT, name STRING, salary DOUBLE);
  -- USING delta  # default table type

INSERT INTO employees
VALUES 
  (1, "Adam", 3500.0),
  (2, "Sarah", 4020.5);

INSERT INTO employees
VALUES
  (3, "John", 2999.3),
  (4, "Thomas", 4000.3);

INSERT INTO employees
VALUES
  (5, "Anna", 2500.0);

INSERT INTO employees
VALUES
  (6, "Kim", 6200.3);

SELECT * FROM employees;

DESCRIBE DETAIL employees;
-- %fs ls 'dbfs:/user/hive/warehouse/employees'

UPDATE employees 
SET salary = salary + 100
WHERE name LIKE "A%";

DESCRIBE HISTORY employees;
--  %fs ls 'dbfs:/user/hive/warehouse/employees/_delta_log'
-- %fs head 'dbfs:/user/hive/warehouse/employees/_delta_log/00000000000000000005.json'

In [0]:
%sql
-- SELECT * FROM employees VERSION AS OF 4;
-- SELECT * FROM employees@v4
-- DELETE FROM employees;
-- SELECT * FROM employees;
-- RESTORE TABLE employees TO VERSION AS OF 5;
-- SELECT * FROM employees;
-- DESCRIBE HISTORY employees
-- DESCRIBE DETAIL employees; 
-- OPTIMIZE employees ZORDER BY id;
-- DESCRIBE DETAIL employees;
-- DESCRIBE HISTORY employees;


In [0]:
%sql
-- %fs ls 'dbfs:/user/hive/warehouse/employees'
-- VACUUM employees;
-- VACUUM employees RETAIN 0 HOURS; -- throws error as the retenstion is 7 by default

-- %fs ls 'dbfs:/user/hive/warehouse/employees'
-- SET spark.databricks.delta.retentionDurationCheck.enabled = false;

-- VACUUM employees RETAIN 0 HOURS
-- SELECT * FROM employees@v1
-- DROP TABLE employees;
-- SELECT * FROM employees
--  %fs ls 'dbfs:/user/hive/warehouse/employees'


In [0]:
%sql
USE CATALOG hive_metastore;

-- CREATE TABLE managed_default
--   (width INT, length INT, height INT);

-- INSERT INTO managed_default VALUES (3 INT, 2 INT, 1 INT);

-- DESCRIBE EXTENDED managed_default;

CREATE TABLE external_default
  (width INT, length INT, height INT)
LOCATION 'dbfs:/mnt/demo/external_default';
  
INSERT INTO external_default
VALUES (3 INT, 2 INT, 1 INT);


-- DROP TABLE managed_default;
-- %fs ls 'dbfs:/user/hive/warehouse/managed_default'

-- DROP TABLE external_default;
-- %fs ls 'dbfs:/mnt/demo/external_default'

In [0]:
%sql
-- CREATE SCHEMA new_default;
-- DESCRIBE DATABASE EXTENDED new_default;

-- USE new_default;

-- CREATE TABLE managed_new_default
--   (width INT, length INT, height INT);
  
-- INSERT INTO managed_new_default VALUES (3 INT, 2 INT, 1 INT);

-- CREATE TABLE external_new_default
--   (width INT, length INT, height INT)
-- LOCATION 'dbfs:/mnt/demo/external_new_default';
  
-- INSERT INTO external_new_default VALUES (3 INT, 2 INT, 1 INT);

DESCRIBE EXTENDED managed_new_default;
DESCRIBE EXTENDED external_new_default;

-- SELECT * FROM external_new_default;

-- DROP TABLE managed_new_default;
-- DROP TABLE external_new_default;

-- %fs ls 'dbfs:/user/hive/warehouse/new_default.db/managed_new_default'
-- %fs ls 'dbfs:/mnt/demo/external_new_default'

In [0]:
%sql
-- CREATE SCHEMA custom 
-- LOCATION 'dbfs:/Shared/schemas/custom.db';

-- DESCRIBE DATABASE EXTENDED custom;

-- USE custom;

-- CREATE TABLE managed_custom
--   (width INT, length INT, height INT);
  
-- INSERT INTO managed_custom
-- VALUES (3 INT, 2 INT, 1 INT);

-- CREATE TABLE external_custom
--   (width INT, length INT, height INT)
-- LOCATION 'dbfs:/mnt/demo/external_custom';
  
-- INSERT INTO external_custom
-- VALUES (3 INT, 2 INT, 1 INT);

-- DESCRIBE EXTENDED managed_custom;
-- DESCRIBE EXTENDED external_custom;

-- DROP TABLE managed_custom;
-- DROP TABLE external_custom;

-- %fs ls 'dbfs:/Shared/schemas/custom.db/managed_custom'
-- %fs ls 'dbfs:/mnt/demo/external_custom'


In [0]:
%sql
-- ### 1. VIEWS  ####

USE CATALOG hive_metastore;

CREATE TABLE IF NOT EXISTS smartphones
(id INT, name STRING, brand STRING, year INT);

INSERT INTO smartphones
VALUES (1, 'iPhone 14', 'Apple', 2022),
      (2, 'iPhone 13', 'Apple', 2021),
      (3, 'iPhone 6', 'Apple', 2014),
      (4, 'iPad Air', 'Apple', 2013),
      (5, 'Galaxy S22', 'Samsung', 2022),
      (6, 'Galaxy Z Fold', 'Samsung', 2022),
      (7, 'Galaxy S9', 'Samsung', 2016),
      (8, '12 Pro', 'Xiaomi', 2022),
      (9, 'Redmi 11T Pro', 'Xiaomi', 2022),
      (10, 'Redmi Note 11', 'Xiaomi', 2021);

-- SHOW TABLES;

CREATE VIEW view_apple_phones
AS  SELECT * 
    FROM smartphones 
    WHERE brand = 'Apple';

SELECT * FROM view_apple_phones;

CREATE TEMP VIEW temp_view_phones_brands
AS  SELECT DISTINCT brand
    FROM smartphones;

SELECT * FROM temp_view_phones_brands;

-- CREATE GLOBAL TEMP VIEW global_temp_view_latest_phones
-- AS SELECT * FROM smartphones
--     WHERE year > 2020
--     ORDER BY year DESC;

SELECT * FROM global_temp.global_temp_view_latest_phones;

-- SHOW TABLES;
SHOW TABLES IN global_temp;

DROP TABLE smartphones;
DROP VIEW view_apple_phones;
DROP VIEW global_temp.global_temp_view_latest_phones;


In [0]:
%run ./Includes/Copy-Datasets

In [0]:
%python
files = dbutils.fs.ls(f"{dataset_bookstore}/customers-json")
display(files)

data_source_uri = "s3://dalhussein-courses/datasets/bookstore/v1/"
dataset_bookstore = 'dbfs:/mnt/demo-datasets/bookstore'
data_catalog = 'hive_metastore'
spark.conf.set(f"dataset.bookstore", dataset_bookstore)
spark.conf.set("fs.s3a.endpoint", "s3.eu-west-3.amazonaws.com")
spark.conf.set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")

In [0]:
%sql
-- SELECT * FROM json.`${dataset.bookstore}/customers-json/export_001.json`;
-- SELECT * FROM json.`${dataset.bookstore}/customers-json/export_*.json`;
-- SELECT *, input_file_name() source_file FROM json.`${dataset.bookstore}/customers-json`;
-- SELECT * FROM text.`${dataset.bookstore}/customers-json`;
-- SELECT * FROM binaryFile.`${dataset.bookstore}/customers-json`;

-- SELECT * FROM csv.`${dataset.bookstore}/books-csv`;
-- CREATE TABLE IF NOT EXISTS books_csv
--   (book_id STRING, title STRING, author STRING, category STRING, price DOUBLE)
-- USING CSV
-- OPTIONS (
--   header = "true",
--   delimiter = ";"
-- )
-- LOCATION "${dataset.bookstore}/books-csv";
-- SELECT * FROM books_csv;
-- DESCRIBE EXTENDED books_csv;


-- %python
-- files = dbutils.fs.ls(f"{dataset_bookstore}/books-csv")
-- display(files)

-- %python
-- (spark.read
--         .table("books_csv")
--       .write
--         .mode("append")
--         .format("csv")
--         .option('header', 'true')
--         .option('delimiter', ';')
--         .save(f"{dataset_bookstore}/books-csv"))


-- %python
-- files = dbutils.fs.ls(f"{dataset_bookstore}/books-csv")
-- display(files)

-- SELECT COUNT(*) FROM books_csv;
-- REFRESH TABLE books_csv;
-- SELECT COUNT(*) FROM books_csv;


-- CREATE TABLE customers AS SELECT * FROM json.`${dataset.bookstore}/customers-json`;
-- DESCRIBE EXTENDED customers;

-- CREATE TABLE books_unparsed AS SELECT * FROM csv.`${dataset.bookstore}/books-csv`;
-- SELECT * FROM books_unparsed;

-- CREATE TEMP VIEW books_tmp_vw
--    (book_id STRING, title STRING, author STRING, category STRING, price DOUBLE)
-- USING CSV
-- OPTIONS (
--   path = "${dataset.bookstore}/books-csv/export_*.csv",
--   header = "true",
--   delimiter = ";"
-- );
-- CREATE TABLE books AS SELECT * FROM books_tmp_vw;
-- SELECT * FROM books;

-- DESCRIBE EXTENDED books;


In [0]:
%sql
-- CREATE TABLE orders AS SELECT * FROM parquet.`${dataset.bookstore}/orders`;
-- SELECT * FROM orders;

-- CREATE OR REPLACE TABLE orders AS SELECT * FROM parquet.`${dataset.bookstore}/orders`;
-- DESCRIBE HISTORY orders;
-- INSERT OVERWRITE orders SELECT * FROM parquet.`${dataset.bookstore}/orders`;
-- DESCRIBE HISTORY orders;
-- INSERT OVERWRITE orders SELECT *, current_timestamp() FROM parquet.`${dataset.bookstore}/orders`;

-- INSERT INTO orders SELECT * FROM parquet.`${dataset.bookstore}/orders-new`;
-- SELECT count(*) FROM orders;


-- CREATE OR REPLACE TEMP VIEW customers_updates AS 
-- SELECT * FROM json.`${dataset.bookstore}/customers-json-new`;

-- MERGE INTO customers c
-- USING customers_updates u
-- ON c.customer_id = u.customer_id
-- WHEN MATCHED AND c.email IS NULL AND u.email IS NOT NULL THEN
--   UPDATE SET email = u.email, updated = u.updated
-- WHEN NOT MATCHED THEN INSERT *;

-- CREATE OR REPLACE TEMP VIEW books_updates
--    (book_id STRING, title STRING, author STRING, category STRING, price DOUBLE)
-- USING CSV
-- OPTIONS (
--   path = "${dataset.bookstore}/books-csv-new",
--   header = "true",
--   delimiter = ";"
-- );

-- SELECT * FROM books_updates;

-- MERGE INTO books b
-- USING books_updates u
-- ON b.book_id = u.book_id AND b.title = u.title
-- WHEN NOT MATCHED AND u.category = 'Computer Science' THEN 
--   INSERT *;

In [0]:
%sql
-- SELECT * FROM customers;
-- DESCRIBE customers;

-- SELECT customer_id, profile:first_name, profile:address:country FROM customers;
-- SELECT from_json(profile) AS profile_struct FROM customers;
-- SELECT profile FROM customers LIMIT 1;

-- ## Get schema of table 
-- CREATE OR REPLACE TEMP VIEW parsed_customers AS
--   SELECT customer_id, from_json(profile, schema_of_json('{"first_name":"Thomas","last_name":"Lane","gender":"Male","address":{"street":"06 Boulevard Victor Hugo","city":"Paris","country":"France"}}')) AS profile_struct
--   FROM customers;  
-- SELECT * FROM parsed_customers;

-- DESCRIBE parsed_customers;
-- SELECT customer_id, profile_struct.first_name, profile_struct.address.country FROM parsed_customers;
-- CREATE OR REPLACE TEMP VIEW customers_final AS SELECT customer_id, profile_struct.* FROM parsed_customers;
-- SELECT * FROM customers_final;

-- SELECT order_id, customer_id, books FROM orders;
-- SELECT order_id, customer_id, explode(books) AS book FROM orders;

-- Collecting Rows
-- SELECT customer_id,
--   collect_set(order_id) AS orders_set,
--   collect_set(books.book_id) AS books_set
-- FROM orders
-- GROUP BY customer_id;

-- Flatten Arrays
-- SELECT customer_id,
--   collect_set(books.book_id) As before_flatten,
--   array_distinct(flatten(collect_set(books.book_id))) AS after_flatten
-- FROM orders
-- GROUP BY customer_id;

-- Join Operations
-- CREATE OR REPLACE VIEW orders_enriched AS
-- SELECT *
-- FROM (
--   SELECT *, explode(books) AS book 
--   FROM orders) o
-- INNER JOIN books b
-- ON o.book.book_id = b.book_id;
-- SELECT * FROM orders_enriched;

-- Set Operations
-- CREATE OR REPLACE TEMP VIEW orders_updates AS SELECT * FROM parquet.`${dataset.bookstore}/orders-new`;

-- SELECT * FROM orders 
-- UNION 
-- SELECT * FROM orders_updates ;

-- SELECT * FROM orders 
-- INTERSECT 
-- SELECT * FROM orders_updates;

-- SELECT * FROM orders 
-- MINUS 
-- SELECT * FROM orders_updates;

-- CREATE OR REPLACE TABLE transactions AS
-- SELECT * FROM (
--   SELECT
--     customer_id,
--     book.book_id AS book_id,
--     book.quantity AS quantity
--   FROM orders_enriched
-- ) PIVOT (
--   sum(quantity) FOR book_id in (
--     'B01', 'B02', 'B03', 'B04', 'B05', 'B06',
--     'B07', 'B08', 'B09', 'B10', 'B11', 'B12'
--   )
-- );
-- SELECT * FROM transactions;


In [0]:
%sql
-- SELECT order_id, books, 
--   FILTER (books, i -> i.quantity >= 2) AS multiple_copies -- having quantity >= 2
-- FROM orders;

-- SELECT order_id, multiple_copies
-- FROM (
--   SELECT
--     order_id,
--     FILTER (books, i -> i.quantity >= 2) AS multiple_copies
--   FROM orders)
-- WHERE size(multiple_copies) > 0;

-- SELECT
--   order_id,
--   books,
--   TRANSFORM (
--     books,
--     b -> CAST(b.subtotal * 0.8 AS INT)
--   ) AS subtotal_after_discount
-- FROM orders;

-- UDF
-- CREATE OR REPLACE FUNCTION get_url(email STRING)
-- RETURNS STRING
-- RETURN concat("https://www.", split(email, "@")[1]);

-- SELECT email, get_url(email) domain FROM customers;
-- DESCRIBE FUNCTION get_url;
-- DESCRIBE FUNCTION EXTENDED get_url;
-- DROP FUNCTION get_url;

In [0]:
%run ./Includes/Copy-Datasets

In [0]:
(spark.readStream
      .table("books")
      .createOrReplaceTempView("books_streaming_tmp_vw")
)


In [0]:
%sql
-- SELECT * FROM books_streaming_tmp_vw;
-- SELECT author, count(book_id) AS total_books FROM books_streaming_tmp_vw GROUP BY author;
-- SELECT * FROM books_streaming_tmp_vw ORDER BY author; -- unsupported

CREATE OR REPLACE TEMP VIEW author_counts_tmp_vw AS (
  SELECT author, count(book_id) AS total_books
  FROM books_streaming_tmp_vw
  GROUP BY author
);


In [0]:
(spark.table("author_counts_tmp_vw")                               
      .writeStream  
      .trigger(processingTime='4 seconds')
      .outputMode("complete")
      .option("checkpointLocation", "dbfs:/mnt/demo/author_counts_checkpoint")
      .table("author_counts")
)

In [0]:
%sql
SELECT *
FROM author_counts

In [0]:
%sql
INSERT INTO books
values ("B19", "Introduction to Modeling and Simulation", "Mark W. Spong", "Computer Science", 25),
        ("B20", "Robot Modeling and Control", "Mark W. Spong", "Computer Science", 30),
        ("B21", "Turing's Vision: The Birth of Computer Science", "Chris Bernhardt", "Computer Science", 35)

In [0]:
%sql
INSERT INTO books
values ("B16", "Hands-On Deep Learning Algorithms with Python", "Sudharsan Ravichandiran", "Computer Science", 25),
        ("B17", "Neural Network Methods in Natural Language Processing", "Yoav Goldberg", "Computer Science", 30),
        ("B18", "Understanding digital signal processing", "Richard Lyons", "Computer Science", 35)

In [0]:
(spark.table("author_counts_tmp_vw")                               
      .writeStream           
      .trigger(availableNow=True) # runs in batch mode. stops when done
      .outputMode("complete")
      .option("checkpointLocation", "dbfs:/mnt/demo/author_counts_checkpoint")
      .table("author_counts")
      .awaitTermination()
)

In [0]:
%sql
SELECT *
FROM author_counts

In [0]:
#### Auto Loader #####
files = dbutils.fs.ls(f"{dataset_bookstore}/orders-raw")
display(files)

In [0]:
(spark.readStream
        .format("cloudFiles")  ## Autoloader
        .option("cloudFiles.format", "parquet")
        .option("cloudFiles.schemaLocation", "dbfs:/mnt/demo/orders_checkpoint")
        .load(f"{dataset_bookstore}/orders-raw")
      .writeStream
        .option("checkpointLocation", "dbfs:/mnt/demo/orders_checkpoint")
        .table("orders_updates")
)

In [0]:
%sql
-- SELECT * FROM orders_updates;
-- SELECT count(*) FROM orders_updates;
-- DESCRIBE HISTORY orders_updates;
DROP TABLE orders_updates;

In [0]:
load_new_data()  ## run twice

In [0]:
files = dbutils.fs.ls(f"{dataset_bookstore}/orders-raw")
display(files)

In [0]:
dbutils.fs.rm("dbfs:/mnt/demo/orders_checkpoint", True)

In [0]:
#### Multi-Hop Architecture ######
files = dbutils.fs.ls(f"{dataset_bookstore}/orders-raw")
display(files)


In [0]:
(spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "parquet")
    .option("cloudFiles.schemaLocation", "dbfs:/mnt/demo/checkpoints/orders_raw")
    .load(f"{dataset_bookstore}/orders-raw")
    .createOrReplaceTempView("orders_raw_temp"))

In [0]:
%sql
-- CREATE OR REPLACE TEMPORARY VIEW orders_tmp AS (
--   SELECT *, current_timestamp() arrival_time, input_file_name() source_file
--   FROM orders_raw_temp
-- );

-- SELECT * FROM orders_tmp;

In [0]:
## Creating Bronze Table

(spark.table("orders_tmp")
      .writeStream
      .format("delta")
      .option("checkpointLocation", "dbfs:/mnt/demo/checkpoints/orders_bronze")
      .outputMode("append")
      .table("orders_bronze"))

In [0]:
%sql
SELECT count(*) FROM orders_bronze;

In [0]:
load_new_data()

In [0]:
############ Creating Static Lookup Table
(spark.read
      .format("json")
      .load(f"{dataset_bookstore}/customers-json")
      .createOrReplaceTempView("customers_lookup"))


In [0]:
%sql
SELECT * FROM customers_lookup

In [0]:
(spark.readStream
  .table("orders_bronze")
  .createOrReplaceTempView("orders_bronze_tmp"))

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW orders_enriched_tmp AS (
  SELECT order_id, quantity, o.customer_id, c.profile:first_name as f_name, c.profile:last_name as l_name,
         cast(from_unixtime(order_timestamp, 'yyyy-MM-dd HH:mm:ss') AS timestamp) order_timestamp, books
  FROM orders_bronze_tmp o
  INNER JOIN customers_lookup c
  ON o.customer_id = c.customer_id
  WHERE quantity > 0)

In [0]:
(spark.table("orders_enriched_tmp")
      .writeStream
      .format("delta")
      .option("checkpointLocation", "dbfs:/mnt/demo/checkpoints/orders_silver")
      .outputMode("append")
      .table("orders_silver"))

In [0]:
%sql
-- SELECT * FROM orders_silver;
SELECT COUNT(*) FROM orders_silver;


In [0]:
load_new_data()

In [0]:
# Creating Gold Table

(spark.readStream
  .table("orders_silver")
  .createOrReplaceTempView("orders_silver_tmp"))

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW daily_customer_books_tmp AS (
  SELECT customer_id, f_name, l_name, date_trunc("DD", order_timestamp) order_date, sum(quantity) books_counts
  FROM orders_silver_tmp
  GROUP BY customer_id, f_name, l_name, date_trunc("DD", order_timestamp)
  )

In [0]:
(spark.table("daily_customer_books_tmp")
      .writeStream
      .format("delta")
      .outputMode("complete")  # or, append
      .option("checkpointLocation", "dbfs:/mnt/demo/checkpoints/daily_customer_books")
      .trigger(availableNow=True) ## micro batches
      .table("daily_customer_books"))

In [0]:
%sql
SELECT * FROM daily_customer_books;

In [0]:
load_new_data()

In [0]:
for s in spark.streams.active:
    print("Stopping stream: " + s.id)
    s.stop()
    s.awaitTermination()